In [9]:
# 데이터 준비
import pandas as pd

campaign_df = pd.read_csv('campaign_master.csv')
class_df = pd.read_csv('class_master.csv')
customer_df = pd.read_csv('customer_master.csv')
use_df = pd.read_csv('use_log.csv')

In [10]:
# 병합
custonmer_join = pd.merge(customer_df, class_df, on='class', how='outer')
custonmer_join = pd.merge(custonmer_join, use_df, on='customer_id', how='left')
custonmer_join = pd.merge(custonmer_join, campaign_df, on='campaign_id', how='left')
custonmer_join.columns.unique()

Index(['customer_id', 'name', 'class', 'gender', 'start_date', 'end_date',
       'campaign_id', 'is_deleted', 'class_name', 'price', 'log_id', 'usedate',
       'campaign_name'],
      dtype='object')

In [11]:
# 결측치 처리
# custonmer_join.isnull().sum()
custonmer_join = custonmer_join.dropna()

custonmer_join.rename(columns={'end_date': 'calc_date'}, inplace=True)
custonmer_join.isnull().sum()

customer_id      0
name             0
class            0
gender           0
start_date       0
calc_date        0
campaign_id      0
is_deleted       0
class_name       0
price            0
log_id           0
usedate          0
campaign_name    0
dtype: int64

In [12]:
# 멤버십 기간 계산(calc_date - start_date)
print(custonmer_join.shape)
custonmer_join['calc_date'] = pd.to_datetime(custonmer_join['calc_date'])
custonmer_join['start_date'] = pd.to_datetime(custonmer_join['start_date'])

custonmer_join['date_difference'] = (custonmer_join['calc_date'] - custonmer_join['start_date']).dt.days
custonmer_join['date_difference'].shape

(29474, 13)


(29474,)

In [13]:
# use_log 데이터를 활용하여 월별 이용횟수(평균, 최대, 최소, 중앙값) 계산
use_df['usedate'] = pd.to_datetime(use_df['usedate'])
use_log = (use_df.set_index('usedate') # 'usedate'를 인덱스로 설정
           .groupby('customer_id') # customer_id별로 그룹화
           .resample('ME')  # 월말로 리샘플링
           .size()  # 크기 계산
           .reset_index(name='count'))

groupby_agg = use_log.groupby('customer_id')['count'].agg(['mean', 'median', 'max', 'min'])
groupby_agg.head(), groupby_agg.shape

(                 mean  median  max  min
 customer_id                            
 AS002855     4.500000     5.0    7    2
 AS008805     4.000000     4.0    8    1
 AS009013     2.000000     2.0    2    2
 AS009373     5.083333     5.0    7    3
 AS015233     7.545455     7.0   11    4,
 (4192, 4))

In [14]:
# use_log 데이터를 활용하여 정기적으로 오는 유저에 대해 routine_flg라는 새로운 칼럼 생성(자주오면 1, 자주 오지 않으면 0)
use_routine = use_log[['customer_id']].drop_duplicates()
use_routine = use_routine.merge(
    groupby_agg['mean'],  # groupby_agg에서 'mean' 컬럼 선택
    on='customer_id',
    how='left'
)
month_mean = use_routine['mean'].mean()
use_routine['routine_flg'] = (use_routine['mean'] >= month_mean).astype(int)
use_routine = use_routine.drop(columns=['mean'])
use_routine.head(), use_routine.shape

(  customer_id  routine_flg
 0    AS002855            0
 1    AS008805            0
 2    AS009013            0
 3    AS009373            0
 4    AS015233            1,
 (4192, 2))

In [15]:
merge_uselog = pd.merge(custonmer_join,
                     pd.merge(groupby_agg, use_routine, on='customer_id', how='left'),
                     on='customer_id', 
                     how='left')
merge_uselog.columns.unique(), merge_uselog.shape

(Index(['customer_id', 'name', 'class', 'gender', 'start_date', 'calc_date',
        'campaign_id', 'is_deleted', 'class_name', 'price', 'log_id', 'usedate',
        'campaign_name', 'date_difference', 'mean', 'median', 'max', 'min',
        'routine_flg'],
       dtype='object'),
 (29474, 19))